In [ ]:
!pip install kaggle --upgrade
!mkdir /root/.kaggle/
!cp '/content/drive/My Drive/kaggle.json' /root/.kaggle/   # kaggl.json위치 지정
!chmod 600 /root/.kaggle/kaggle.json
!kaggle datasets download -d dhuh137/hmdb51
!unzip hmdb51.zip

In [ ]:
import os
import numpy as np

for foldername in os.listdir('/content/HMDB51_NUMPY/fold_1/training/0'):
  for filename in os.listdir('/content/HMDB51_NUMPY/fold_1/training/0' + '/'  + foldername):
    temp = np.load('/content/HMDB51_NUMPY/fold_1/training/0' + '/'  + foldername + '/' + filename)
    if temp.shape != (11,64,64,3):
      print(filename)
      os.remove('/content/HMDB51_NUMPY/fold_1/training/0' + '/'  + foldername + '/' + filename)

In [ ]:
import numpy as np
import keras

class DataGenerator(keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, list_IDs, labels, batch_size=32, dim=(32,32,32), n_channels=1,
                 n_classes=10, shuffle=True):
        'Initialization'
        self.dim = dim
        self.batch_size = batch_size
        self.labels = labels
        self.list_IDs = list_IDs
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.list_IDs) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Find list of IDs
        list_IDs_temp = [self.list_IDs[k] for k in indexes]

        # Generate data
        X, y = self.__data_generation(list_IDs_temp)

        return X, y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, list_IDs_temp):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization
        X = np.empty((self.batch_size, *self.dim, self.n_channels))
        y = np.empty((self.batch_size), dtype=int)

        # Generate data
        for i, ID in enumerate(list_IDs_temp):
            # Store sample
              
              X[i,] = np.load(ID)

              # Store class
              y[i] = self.labels[ID]

        return X, keras.utils.to_categorical(y, num_classes=self.n_classes)




In [ ]:
import numpy as np
import os
from keras.models import Sequential


# Parameters
params = {'dim': (11,64,64),
          'batch_size': 2,
          'n_classes': 51,
          'n_channels': 3,
          'shuffle': True}

# Datasets
#partition = {'train': ['id-1', 'id-2', 'id-3'], 'validation': ['id-4']}

DATA_PATH = '/content/HMDB51_NUMPY/fold_1/training/0'
labels = {}
partitions = {'train':[]}
for classname in os.listdir(DATA_PATH):
  for video in os.listdir(DATA_PATH + '/' + classname):
    id = DATA_PATH + '/' + classname + '/' + video
    partitions['train'].append(id)
    labels[id] = classname



# Generators
training_generator = DataGenerator(partitions['train'], labels, **params)
#validation_generator = DataGenerator(partition['validation'], labels, **params)




In [ ]:
import os
os.chdir('/content/drive/My Drive/종합설계/myslowfast')

In [ ]:
from util.utils import get_optimizer
from util import opts
from model.model import SlowFast_body
from util.Callback import create_callbacks

In [ ]:
opt = opts.opts()
print(opt.root_path)

In [ ]:
opt = opts.opts()
model = SlowFast_body(input_shape = (11,64,64,3),num_classes=51)
callback = create_callbacks(opt,len(training_generator),model)
optimizer = get_optimizer(opt)

In [ ]:
import tensorflow as tf

model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
# Train model on dataset
with tf.device('/device:GPU:0'):
  hist = model.fit_generator(generator=training_generator,steps_per_epoch=len(training_generator),epochs=50,callbacks=callback)

model.save_weights(os.path.join(os.path.join(opt.root_path, opt.result_path), 'trained_weights_final.h5'))

    

In [ ]:
import os
import math
import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras.layers import Input
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.callbacks import TensorBoard, EarlyStopping, ModelCheckpoint
from tensorflow.keras.utils import multi_gpu_model
from model import nets
from opts import parse_opts
from utils import get_optimizer, SGDRScheduler_with_WarmUp, TrainPrint, PrintLearningRate, ParallelModelCheckpoint
from dataset.dataset import DataGenerator


def create_callbacks(opt, steps_per_epoch, model=None):
    log_dir = os.path.join(opt.root_path, opt.log_dir)
    if not os.path.exists(log_dir):
        os.mkdir(log_dir)
    tensorboard = TensorBoard(log_dir=log_dir, write_graph=True)

    result_path = os.path.join(opt.root_path, opt.result_path)
    if not os.path.exists(result_path):
        os.mkdir(result_path)

    if model is not None:
        checkpoint = ParallelModelCheckpoint(model, os.path.join(result_path, 'ep{epoch:03d}-val_acc{val_acc:.2f}.h5'),
                                    monitor='val_acc', save_weights_only=True, save_best_only=True, period=1)
    else:
        checkpoint = ModelCheckpoint(os.path.join(result_path, 'ep{epoch:03d}-val_acc{val_acc:.2f}.h5'),
                                    monitor='val_acc', save_weights_only=True, save_best_only=True, period=1)
    early_stopping = EarlyStopping(monitor='val_acc', min_delta=0, patience=10)
    learning_rate_scheduler = SGDRScheduler_with_WarmUp(0, opt.lr, steps_per_epoch, lr_decay=opt.lr_decay, 
                                                        cycle_length=opt.cycle_length, multi_factor=opt.multi_factor,
                                                        warm_up_epoch=opt.warm_up_epoch)
    #training_print = TrainPrint(steps_per_epoch, opt.epochs)
    print_lr = PrintLearningRate()

    return [tensorboard, learning_rate_scheduler, print_lr, checkpoint, early_stopping]
    


def train(opt):
    K.clear_session()
    video_input = Input(shape=(None, None, None, 3))
    model = nets.network[opt.network](video_input, num_classes=opt.num_classes)
    print("Create {} model with {} classes".format(opt.network, opt.num_classes))

    if opt.pretrained_weights is not None:
        model.load_weights(opt.pretrained_weights)
        print("Loading weights from {}".format(opt.pretrained_weights))

    optimizer = get_optimizer(opt)

    train_data_generator = DataGenerator(opt.data_name, opt.video_path, opt.train_list, opt.name_path, 
                                        'train', opt.batch_size, opt.num_classes, True, opt.short_side, 
                                        opt.crop_size, opt.clip_len, opt.n_samples_for_each_video)
    val_data_generator = DataGenerator(opt.data_name, opt.video_path, opt.val_list, opt.name_path, 'val', 
                                        opt.batch_size, opt.num_classes, False, opt.short_side, 
                                        opt.crop_size, opt.clip_len, opt.n_samples_for_each_video)
    
    
    callbacks = create_callbacks(opt, max(1, train_data_generator.__len__()), model)

    if len(opt.gpus) > 1:
        print('Using multi gpus')
        parallel_model = multi_gpu_model(model, gpus=len(opt.gpus))
        parallel_model.compile(optimizer=optimizer, loss=categorical_crossentropy, metrics=['accuracy'])
        parallel_model.fit_generator(train_data_generator, steps_per_epoch=max(1, train_data_generator.__len__()),
                            epochs=opt.epochs, validation_data=val_data_generator, validation_steps=max(1, val_data_generator.__len__()),
                            workers=opt.workers, callbacks=callbacks)
    else:
        model.compile(optimizer=optimizer, loss=categorical_crossentropy, metrics=['accuracy'])
        model.fit_generator(train_data_generator, steps_per_epoch=max(1, train_data_generator.__len__()),
                            epochs=opt.epochs, validation_data=val_data_generator, validation_steps=max(1, val_data_generator.__len__()),
                            workers=opt.workers, callbacks=callbacks)
    model.save_weights(os.path.join(os.path.join(opt.root_path, opt.result_path), 'trained_weights_final.h5'))

    
if __name__=="__main__":
    opt = parse_opts()
    print(opt)
    if len(opt.gpus) > 1:
        os.environ['CUDA_VISIBLE_DEVICES'] = ",".join(map(str, opt.gpus))
    train(opt)